# Iteration 1: Acute Stroke Unit Arrivals

## Plain English Summary

The first step in coding a `simpy` model is typically coding the arrivals to the system.  In the Acute Stroke Unit (ASU) part of the model accepts four patient type arrivals model: strokes, transient ischemic attacks (TIA; these are strokes where symptoms resolve rapidly), complex neurological, and other (medical outliers).



## Notebook imports

In [1]:
from IPython.display import Markdown, display

## LLM Prompt

In [2]:
display(Markdown("inputs/01_iteration.md"))

### Main command

Code a discrete-event simulation model of an Acute Stroke Unit (ACU) in python 3.10 and simpy 4. 

Please code the full model specified. Do not return a simplified version. Show all code.

### General model logic

All time units in the model are in days.

The model should count the total number of patient arrivals.  The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifier by 1.  The model should also maintain counts of the number of patients broken down by their type.

Patients to arrive at the ACU from four different sources: Stroke, TIA, Complex Neurological, or Other. Each source had a different inter-arrival time distribution.  After patients arrive they immediately leave the model.

All patient types must have their own generator function.

The model should print out useful information after each event.

The model should include a user settable run length. This defaults to 5 years.


### Simulation inputs

the interarrival time distributions and parameters of patients are dependent on patient type. For each distribution time is measured in days.

Stroke = Exponential: 1.2
TIA = Exponential: 9.3
Complex Neurological = Exponential: 3.6
Other = Exponential: 3.2


### simulation methodology

numpy should be use for sampling.

each interarrival distribution should have its own numpy.random.Generator object.

## Date and time of input to LLM

> 1545 9th February 2024

## Observations on the LLM Output

* The model code is adding each patient type into a simpy container after they have arrived. It is unclear exactly why it is doing this and it will be removed in iteration  in favour of tracking counts of patient types.  E.g.

```python
self.stroke_arrival = simpy.Container(env, init=0)
```

* The run length 365 * 5 in the correct units and the correct calculation of 1825

* The design to use `numpy` random number generators did not use the request `numpy.random.Generator` object and uses old cold instead (with a single stream). E.g. for the main stroke patient the sampling uses:
```python
interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
```

## Sources reported by the LLM

* https://youtu.be/qe3sms-TqD0
* https://stackoverflow.com/questions/74181096/exception-during-discrete-event-execution
* https://www.researchgate.net/publication/274084058_An_Agent-based_Discrete_Event_Simulation_Approach_for_Modeling_Large-Scale_Disaster_Evacuation_Network
* https://apps.dtic.mil/sti/tr/pdf/ADA567384.pdf

## LLM Output

In [3]:
import simpy
import numpy as np

# Define the discrete-event simulation model
class AcuteStrokeUnit:
    def __init__(self, env, run_length=1825):  # 5 years in days
        self.env = env
        self.stroke_arrival = simpy.Container(env, init=0)
        self.tia_arrival = simpy.Container(env, init=0)
        self.neuro_arrival = simpy.Container(env, init=0)
        self.other_arrival = simpy.Container(env, init=0)
        self.stroke_interarrival_mean = 1.2
        self.tia_interarrival_mean = 9.3
        self.neuro_interarrival_mean = 3.6
        self.other_interarrival_mean = 3.2
        self.run_length = run_length
        self.patient_count = 0

    def stroke_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.stroke_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.stroke_arrival.put(1)
            self.patient_count += 1
            print(f"Stroke patient {self.patient_count} arrived at {self.env.now} days")

    def tia_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.tia_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.tia_arrival.put(1)
            self.patient_count += 1
            print(f"TIA patient {self.patient_count} arrived at {self.env.now} days")

    def neuro_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.neuro_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.neuro_arrival.put(1)
            self.patient_count += 1
            print(f"Complex Neuro patient {self.patient_count} arrived at {self.env.now} days")

    def other_patient_generator(self):
        while True:
            interarrival_time = np.random.exponential(self.other_interarrival_mean)
            yield self.env.timeout(interarrival_time)
            self.other_arrival.put(1)
            self.patient_count += 1
            print(f"Other patient {self.patient_count} arrived at {self.env.now} days")

# Create the simulation environment
env = simpy.Environment()

# Initialize the Acute Stroke Unit model
acu = AcuteStrokeUnit(env)

# Start the patient generators
env.process(acu.stroke_patient_generator())
env.process(acu.tia_patient_generator())
env.process(acu.neuro_patient_generator())
env.process(acu.other_patient_generator())

# Run the simulation
env.run(until=acu.run_length)


Other patient 1 arrived at 0.5358255896729797 days
Other patient 2 arrived at 1.6425878963669742 days
Stroke patient 3 arrived at 2.0549851165126682 days
TIA patient 4 arrived at 2.2849673511832824 days
Stroke patient 5 arrived at 3.698926522192626 days
Stroke patient 6 arrived at 4.249924206911271 days
Other patient 7 arrived at 4.551831828798614 days
Complex Neuro patient 8 arrived at 5.838769778247467 days
Stroke patient 9 arrived at 6.59358255788238 days
Complex Neuro patient 10 arrived at 8.078920599823997 days
Other patient 11 arrived at 8.586184230296148 days
Stroke patient 12 arrived at 9.853582852056762 days
Stroke patient 13 arrived at 10.894597586628253 days
Stroke patient 14 arrived at 12.242710121167113 days
Stroke patient 15 arrived at 13.276620596273982 days
Stroke patient 16 arrived at 13.770942822356869 days
Other patient 17 arrived at 13.98782391531779 days
Other patient 18 arrived at 14.483224484395409 days
Stroke patient 19 arrived at 16.070349292501078 days
Stroke 

## Testing

* **Extreme value test 1**: TIA, Compelex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods

### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [4]:
M = 10_000_000

In [5]:
def test_1(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = 1.2
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [6]:
test_1(M)

Stroke patient 1 arrived at 0.7115253176656521 days
Stroke patient 2 arrived at 1.526047949302964 days
Stroke patient 3 arrived at 3.4004839058009093 days
Stroke patient 4 arrived at 5.0554127171673855 days
Stroke patient 5 arrived at 5.684008810782038 days
Stroke patient 6 arrived at 8.625021043516133 days
Stroke patient 7 arrived at 8.654078518921478 days
Stroke patient 8 arrived at 8.664792470795673 days
Stroke patient 9 arrived at 9.145827740939833 days
Stroke patient 10 arrived at 9.766463540195145 days
Stroke patient 11 arrived at 13.278792961435858 days
Stroke patient 12 arrived at 18.159710758596816 days
Stroke patient 13 arrived at 18.934833998784335 days
Stroke patient 14 arrived at 20.76340270692313 days
Stroke patient 15 arrived at 21.455998530147042 days
Stroke patient 16 arrived at 22.29068651255279 days
Stroke patient 17 arrived at 23.291544294238427 days
Stroke patient 18 arrived at 23.46505720701974 days
Stroke patient 19 arrived at 25.57835458584123 days
Stroke patien

### Extreme value test 2

* The code is again modified so that all patient type also have a mean inter-arrival rate of $M$

**Result: PASS**

In [7]:
def test_2(large_number):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env)

    #set ASU arrival rates
    asu.stroke_interarrival_mean = large_number
    asu.tia_interarrival_mean = large_number
    asu.neuro_interarrival_mean = large_number
    asu.other_interarrival_mean = large_number
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(asu.patient_count)


In [8]:
test_2(M)

0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [9]:
TEN_DAYS = 10

In [10]:
def test_3(run_length):
    # Create the simulation environment
    env = simpy.Environment()
    
    # Initialize the Acute Stroke Unit model
    asu = AcuteStrokeUnit(env, run_length=run_length)
    
    # Start the patient generators
    env.process(asu.stroke_patient_generator())
    env.process(asu.tia_patient_generator())
    env.process(asu.neuro_patient_generator())
    env.process(asu.other_patient_generator())
    
    # Run the simulation
    env.run(until=asu.run_length)

    print(env.now)
    


In [11]:
test_3(TEN_DAYS)

Stroke patient 1 arrived at 0.8242573098365682 days
Other patient 2 arrived at 1.2899623969059186 days
Stroke patient 3 arrived at 1.8303763061051614 days
Other patient 4 arrived at 2.1517941864170407 days
TIA patient 5 arrived at 2.705951355614795 days
Stroke patient 6 arrived at 2.7939260000011714 days
Other patient 7 arrived at 3.8683696965687187 days
Complex Neuro patient 8 arrived at 3.8730690483887957 days
Complex Neuro patient 9 arrived at 4.2845512309502825 days
Stroke patient 10 arrived at 4.455975708347359 days
Stroke patient 11 arrived at 4.723586967187831 days
Stroke patient 12 arrived at 5.951824776467206 days
Stroke patient 13 arrived at 6.927025939723766 days
Stroke patient 14 arrived at 7.199113051485439 days
Other patient 15 arrived at 7.343281744823884 days
Stroke patient 16 arrived at 9.647024989422547 days
Stroke patient 17 arrived at 9.884352538960208 days
Stroke patient 18 arrived at 9.986470563631954 days
10
